# Load Libraries

In [102]:
# pip3 install s3fs
from sagemaker import Session
import sagemaker
import boto3
import re
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import os

from sagemaker import get_execution_role

role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# Load File

In [103]:
bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'customer'
file_name = 'customer_profile.csv'

data_location = 's3://{}/{}/{}'.format(bucket_name, prefix_name, file_name)

df = pd.read_csv(data_location)

print("---------------------------------")
print(f"Shape of dataframe '{df.shape}':")

df.to_csv(file_name, index=False, header=True)

---------------------------------
Shape of dataframe '(10127, 7)':


# Featuring Engeniring

In [104]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

from sagemaker import Session
import sagemaker
from sagemaker import get_execution_role
from sagemaker.workflow.parameters import ( 
 ParameterInteger,
 ParameterString,
 ParameterFloat)

In [105]:
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)

In [106]:
framework_version = "1.0-1"
sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    role=role,
    instance_type="ml.m5.large",
    instance_count=1,
    base_job_name='mlops-customer-feature-eng'
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [107]:
bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'customer/notebook'
file_name = 'customer_profile.csv'

In [108]:
input_source = 's3://{}/{}/{}'.format(bucket_name, prefix_name, file_name)
data_location = 's3://{}/{}/{}'.format(bucket_name, prefix_name, 'requirements.txt')

# Load Files
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix_name, 'requirements.txt')).upload_file('requirements.txt')
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix_name, file_name)).upload_file(file_name)

print("---------------------------------")
print(f"bucket_name '{bucket_name}':")
print(f"prefix_name '{prefix_name}':")
print(f"input_source '{input_source}':")
print(f"data_location '{data_location}':")

---------------------------------
bucket_name 'eliezerraj-908671954593-dataset':
prefix_name 'customer/notebook':
input_source 's3://eliezerraj-908671954593-dataset/customer/notebook/customer_profile.csv':
data_location 's3://eliezerraj-908671954593-dataset/customer/notebook/requirements.txt':


In [109]:
processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=input_source,
                        destination="/opt/ml/processing/input"),
        ProcessingInput(source=data_location,
                        destination="/opt/ml/processing/input/req/")
    ],
    outputs=[
        ProcessingOutput(output_name="train",
                         source="/opt/ml/processing/train",
                         destination=f"s3://{bucket_name}/{prefix_name}/output/train"),
    ],
    code=f"feature.py",
)

INFO:sagemaker:Creating processing-job with name mlops-customer-feature-eng-2024-04-26-00-57-56-282


............................Collecting imbalanced-learn
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 7.6 MB/s eta 0:00:00
-------------------- dataframe data  ----------------------------
shape :  (10127, 7)
-------------------- df_customera  ----------------------------
   CLIENTNUM  Customer_Age  ... Marital_Status  Income_Category
0  768805383            45  ...        Married      $60K - $80K
1  818770008            49  ...         Single   Less than $40K
2  713982108            51  ...        Married     $80K - $120K
[3 rows x 7 columns]
-------------------- df_training  ----------------------------
   Customer_Age  ...  Income_Category_Quality
0            45  ...                        3
1            49  ...                        1
2            51  ...                        4
[3 rows x 4 columns]
## Processing completed. Exiting.

